In [3]:
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import torch as to
import torch.nn as nn 
import torch.nn.functional as func
from torch.utils import data
from torch.utils.data import DataLoader
import random 
import time 
import torch.nn.utils.rnn as rnnutils
from sklearn.model_selection import train_test_split
import torchviz
from torchsummary import summary
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score 
from itertools import tee
import hashlib

### Info 
This notebook is here for preliminary DL experiments
- We are doing mbti classification(one model for each trait)
- All cells should be run sequentialy from top to bottom
- Last cell with the for loop is the main part

In CELL METADATA are mostly constants that shouldn't be change except for: 
- DATASET: contains the path to the dataset
- FOLDS: contains the path to fold(each author should have a number assigned to him that represents a fold)
- AUTHORS: should contain a path to a dataframe about authors 
- no_of_folds: should correspond to the number of different folds 

In CELL PARAMETERS are mostly customizable variables; Most are self-explanatory. 

Results of the experiments are saved in results.csv file.

### Arhitecture
The arhitecture that is specified contains n convolutional filters. Each filters is applied to a verticaly 
arranged vector of authors sentences. After the convolution we get n differently sized vectors. N corresponds to the number of filters we used. On each filter a maxpool is applied. And it all goes into one fc with 2 neurons. All activation functions are the default ones (for pytorch it is sigmoid). 

F1 is the measure of choice(binary one, not micro or macro). 

In the results dataframe there is for each combination of hyperparameters one input. For each fold and each trait the best arhitecture is picked and applied to the test set. Thus, for a lot of arhitectures there are no results for the test set. The best arhitecture and the best models parameters are picked in regard to the F1. If there is no change in recall across several epochs the training is stoped.

### Problems and solutions
-> It looks like the model is overfitting
0> We should downsize the arhitecture a bit 

-> In the first epoch the recall is 1 for a lot of arhitectures and learning rates 
0> We are going to balance the dataset, the model should have a harder time and might even learn something 
0> Macro should be used and checked for both labels 

?> Maybe we should try batch normalization as berts embeddings have only positive weights constantly(not sure if this is true) 

### Future 
Check the file baseline_take2.ipynb for improvements and following experiments 

In [4]:
#CELL METADATA 
DATASET = '/home/mbosnjak/Datasets/embedded_comments_last_100_min_20com_per_auth_w_wc_10_200_no_mbti.csv'
FOLDS = '/home/mbosnjak/Datasets/comments_last_100_min_20com_per_auth_w_wc_10_200.csv.folds.csv'
AUTHORS = '/home/mbosnjak/Datasets/author_profiles.csv'
RELEVANT_DIFFERENCE = 0.00001
NUMBER_OF_CLASSES = 2
STATE = 156
no_of_folds = 5

mbti_labels = ['introverted', 'intuitive', 'thinking', 'perceiving']
MODELS_PREFORMANCE_COLUMNS = ['val_f1', 'val_precision','val_recall',\
                              'test_f1', 'test_precision','test_recall',\
                              'epoch']
MODELS_PARAMETER_COLUMNS = ['mbti_trait', 'fold', 'learning_rate', 'batch_size',\
                           'kernels_count', 'sentences_count' ]
MODELS_META_DATA_COLUMNS = ['models_name', 'hash_id']
to.manual_seed(STATE)

In [13]:
# CELL PARAMETERS
batch_sizes = [1, 4, 16]
learning_rates = [0.001, 0.01, 0.0001, 0.00001, 0.1]
kernel_sizes = [16, 24, 32]
sentences_counts = [2, 4]
n_epochs = 30
use_GPU = True
#TODO: this is sentences per author not comments per author, quick fix is max size 200*100 (200 max sentences by comment * 100 comments)
comments_per_author = 10000
#comments_per_author = 20
cuda_device = to.device("cuda") if use_GPU else None
validation_p = .2
print_status_batch = 1000
max_constant_recall = 3

In [6]:
input_df = pd.read_csv(DATASET)
#input_df = pd.read_csv(DATASET, nrows = 2000) # TODO: remove nrows
folds_df = pd.read_csv(FOLDS, usecols =['author', 'fold'])
authors_profiles_df = pd.read_csv(AUTHORS)
authors_with_mbti = list(authors_profiles_df[authors_profiles_df.introverted.notnull()]['author'])

In [7]:
results = pd.DataFrame(data=None, columns=MODELS_META_DATA_COLUMNS+MODELS_PREFORMANCE_COLUMNS+MODELS_PARAMETER_COLUMNS)
for name in MODELS_PREFORMANCE_COLUMNS:
    results = results.astype({name: 'float64'})

In [8]:
#TODO: add comment delimiter -> to recognize comments end and beginning
def merger(comments):
    npad = [(0, max(sentences_counts)), (0, 0)]
    carpet = comments[:comments_per_author].as_matrix(columns=comments.columns[1:1025])
    carpet_padded = np.pad(carpet, pad_width=npad, mode='constant', constant_values=0)
    #print(f'before {carpet.shape} - after {carpet_padded.shape}')
    return to.tensor(carpet_padded)

def get_input_output(data_authors, target):
    data_df = input_df[input_df['author'].isin(data_authors)]
    input_x_df = data_df.groupby(['author']).apply(merger)
    authors = list(input_x_df.index.values)
    output = authors_profiles_df[authors_profiles_df['author'].isin(authors)][target]
    return input_x_df.tolist(), output.tolist()

def set_hash(index):
    global results
    identifier = hashlib.md5(''.join([str(x) for x in results[MODELS_PARAMETER_COLUMNS].iloc[index]]).encode('utf-8')).hexdigest()
    results.at[index, 'hash_id'] = identifier
    return identifier

def create_new_model_input(mbti_trait, fold, learning_rate, batch_size,\
                           kernels_count, sentences_count):
    global results
    results = results.append({'mbti_trait' : mbti_trait, 'fold' : fold, 'learning_rate' : learning_rate, 'batch_size' : batch_size, 'kernels_count' : kernels_count, 'sentences_count' : sentences_count}, ignore_index=True)
    return set_hash(len(results)-1)

def get_checkpoints_name(models_identifier):
    return "save_"+models_identifier + ".pt"

def update_models_val_results(models_identifier, val_f1, val_precision, val_recall, epoch):
    global results
    results.loc[results['hash_id'] == models_identifier, 'val_f1'] = val_f1
    results.loc[results['hash_id'] == models_identifier, 'val_precision'] = val_precision
    results.loc[results['hash_id'] == models_identifier, 'val_recall'] = val_recall
    results.loc[results['hash_id'] == models_identifier, 'epoch'] = epoch
    
def update_models_test_results(models_identifier, test_f1, test_precision, test_recall):
    global results
    results.loc[results['hash_id'] == models_identifier, 'test_f1'] = test_f1
    results.loc[results['hash_id'] == models_identifier, 'test_precision'] = test_precision
    results.loc[results['hash_id'] == models_identifier, 'test_recall'] = test_recall
    
def get_best_models_data(target, fold):
    return results.loc[results[(results.mbti_trait == target) & (results.fold == fold)]['val_f1'].idxmax()]
    

In [ ]:
create_new_model_input('dfsfdsf',333,3,223333,1,13)

In [9]:
class ConvoCarpet(nn.Module):
    def __init__ (self, embedding_size = 1024, kernels_count=64, sentences_count=2):
        super(ConvoCarpet, self).__init__()
        
        self.conv_layer = nn.Conv2d(1, kernels_count, [sentences_count, embedding_size])
        self.pool_layer = nn.AdaptiveMaxPool2d((1, None))
        self.fc_layer = nn.Linear(kernels_count, NUMBER_OF_CLASSES)
        
    def forward(self, input_batch):
        conv_output = func.relu(self.conv_layer(input_batch))
        maxpool_output = self.pool_layer(conv_output)
        maxpool_output = maxpool_output.flatten(start_dim=1)
        linear_output = self.fc_layer(maxpool_output)
        #return func.log_softmax(linear_output)
        return linear_output


### FLOW

In [10]:
def data_preparation(target, fold):
    test_data_authors = list(folds_df[(folds_df['fold'] == fold) & (folds_df['author'].isin(authors_with_mbti))]['author'])
    train_data_authors = list(folds_df[(folds_df['fold'] != fold) & (folds_df['author'].isin(authors_with_mbti))]['author'])
    train_input, train_output = get_input_output(train_data_authors, target)
    test_input, test_output = get_input_output(test_data_authors, target)

    train_input, val_input, train_output, val_output = train_test_split(train_input, train_output, test_size=validation_p, random_state=STATE)
    assert len(train_input) == len(train_output)
    assert len(test_input) == len(test_output)
    assert len(val_input) == len(val_output)
    return train_input, train_output, val_input, val_output, test_input, test_output
        

def chunks(l, n):
    return [l[i:i + n] for i in range(0, len(l), n)]  

def create_minibatches(data_X, data_y, minibatch_size, cuda_dev):
    for idx_list in chunks(range(len(data_X)), minibatch_size):
        data_X_idx = [data_X[index] for index in idx_list]
        data_y_idx = [int(data_y[index]) for index in idx_list]
        Xy = zip(data_X_idx, data_y_idx) # zip to permute the labels as well when sorting by length
        Xy = sorted(Xy, key = lambda x: len(x[0]), reverse = True)
        minibatch_X, minibatch_y = [list(x) for x in zip(*Xy)] # undo the zip thing to get separate variables for data and labels
        minibatch_X = rnnutils.pad_sequence(minibatch_X, batch_first=True, padding_value = 0) 
        minibatch_X = minibatch_X.unsqueeze(1)
        minibatch_y = to.tensor(minibatch_y)
        if cuda_dev is not None:
            minibatch_X = minibatch_X#.to(device=cuda_dev, dtype=to.float)
            minibatch_y = minibatch_y.to(device=cuda_dev, dtype=to.long)

        yield((minibatch_X, minibatch_y))   


In [11]:
def calcMeasures(total_logits, total_true, threshold):
    total_preds = []
    sm_func = to.nn.Softmax(dim=1)
    for minibatch_logits in total_logits:
        sm_res = sm_func(minibatch_logits)
        minibatch_preds = sm_res[:,1] > threshold
        total_preds.append(minibatch_preds)
    total_preds = to.cat(total_preds)
    total_true = to.cat(total_true)
    F1 = f1_score(total_true.cpu(), total_preds.cpu(), pos_label = 1)
    precision = precision_score(total_true.cpu(), total_preds.cpu())
    recall = recall_score(total_true.cpu(), total_preds.cpu())
    return F1, precision, recall

def apply_model(model, loss_func, data_loader):
    model.eval()
    total_loss = 0
    predicted = []
    true_output = []
    minibatch_loss = 0
    with to.no_grad():
        n_batch = 0
        for inputs, labels in data_loader:
            n_batch += 1
            minibatch_logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss_func(minibatch_logits, labels)
            
            total_loss += minibatch_loss
            predicted.append(minibatch_logits)
            true_output.append(labels)
            
    return total_loss, predicted, true_output

def optimize_models_params(trait, fold, train_input, train_output, val_input, val_output):
    print('Start training ...')
    
    loss = nn.CrossEntropyLoss()

    for batch_size in batch_sizes:
        for kernel_size in kernel_sizes:
            for sent_size in sentences_counts:
                for learning_rate in learning_rates:                
                    
                    train_loader = create_minibatches(train_input, train_output, batch_size, cuda_device)
                    val_loader = create_minibatches(val_input, val_output, batch_size, cuda_device)
        
                    print('-'*10)
                    print(f'Starting straining for BS: {batch_size} KS: {kernel_size} SS: {sent_size} LR: {learning_rate}')
                    models_identifier = create_new_model_input(trait, fold, learning_rate, batch_size, kernel_size, sent_size)
                    
                    model = ConvoCarpet(kernels_count=kernel_size, sentences_count=sent_size)
                    if use_GPU:
                          model.to(cuda_device)

                    optimizer = to.optim.Adam(model.parameters(), learning_rate, amsgrad = True)

                    train_model(train_loader, val_loader, models_identifier, model, loss, optimizer)
                    results.to_csv('results.csv')
    

def train_model(train_loader, val_loader, models_identifier, model, loss, optimizer):
    best_f1_val = 0
    last_recall = 0
    counter = 0
    
    for epoch in range(0, n_epochs):
        epoch_start = time.time()
        model.train()
        total_loss = 0
        n_batch = 0
        train_loader, train_loader_backup = tee(train_loader)
        for inputs, labels in train_loader_backup:

            n_batch += 1
            #print(f"Current batch is {n_batch}")

            optimizer.zero_grad()

            logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss(logits, labels)
            total_loss += minibatch_loss
            minibatch_loss.backward()
            optimizer.step()
            if(n_batch % print_status_batch == 0):
                print("-----------------------")
                print(f"Train - Epoch {epoch} - batch {n_batch}, batch loss is {minibatch_loss:.6f}, total loss is {total_loss:.6f}")
                print("-----------------------")

                
        print("-----------------------")
        
        val_loader, val_loader_backup = tee(val_loader)
        val_loss, val_logits, true_val = apply_model(model, loss, val_loader_backup)
        val_f1_score, val_prec, val_recal = calcMeasures(val_logits, true_val, 0.5)
        
        
        print(f'Epoch end: {time.time()-epoch_start}')
        print(f'At the end of epoch {epoch} loss is: {total_loss:.5f}')
        print(f'Validation loss: {val_loss:.5f} - F1 score: {val_f1_score:.5f} - precision: {val_prec} - recall: {val_recal}')
        print("-----------------------")
        
        if (abs(val_recal - last_recall) <= RELEVANT_DIFFERENCE):
            counter += 1
        else:
            last_recall = val_recal
            counter = 0
        
        if(counter > max_constant_recall):
            return
        
        if (val_f1_score > best_f1_val):
            update_models_val_results(models_identifier, val_f1_score, val_prec, val_recal, epoch)
            best_f1_val = val_f1_score
        # save a checkpoint
            best_checkpoint_filename = get_checkpoints_name(models_identifier)
            to.save({
              "models_identifier" : models_identifier, 
              "model_state_dict" : model.state_dict(),
              "optimizer_state_dict" : optimizer.state_dict()
            }, best_checkpoint_filename)
    

In [14]:
for target in mbti_labels:
    print(f"Creating model for target: {target}")
    for fold in range(0, no_of_folds):
        
        print(f'Processing fold {fold}...')
        
        train_input, train_output, val_input, val_output, test_input, test_output = data_preparation(target, fold)
        
        optimize_models_params(target, fold, train_input, train_output, val_input, val_output)
        
        
        best_models = get_best_models_data(target, fold)
        #Recreate model
        models_identifier = best_models['hash_id']
        model = ConvoCarpet(kernels_count=best_models['kernels_count'] , sentences_count=best_models['sentences_count'])
        if use_GPU:
              model.to(cuda_device)

        checkpoint = to.load(get_checkpoints_name(models_identifier))
        model.load_state_dict(checkpoint["model_state_dict"])
        loss = nn.CrossEntropyLoss()

        
        print("."*15)
        print("."*15)
        
        print(f'Apply best model to test for {target} on fold {fold}.')
        test_loader = create_minibatches(test_input, test_output, 1, cuda_device)
        test_loss, test_logits, test_true = apply_model(model, loss, test_loader)
        test_f1_score, test_pred, test_recal = calcMeasures(test_logits, test_true, 0.5)
        print(f'Test loss: {test_loss:.5f} - F1 score: {test_f1_score:.2f} precision: {test_pred} recall: {test_recal}')
        
        update_models_test_results(models_identifier,test_f1_score, test_pred, test_recal)
        with open(f'{target}_{fold}_{models_identifier}_predictions', 'w+') as f:
            for pre_l, true_l in zip(test_logits, test_true):
                f.write(f'{pre_l}, {true_l}')
        results.to_csv('results.csv')
        
        ##Save best models data
        
        print(f"+++ Finished with training and testing model for {target} on fold {fold}. +++")
        
        print("."*15)
        print("."*15)

Creating model for target: introverted
Processing fold 0...


/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


Start training ...
----------
Starting straining for BS: 1 KS: 16 SS: 2 LR: 0.001
-----------------------
Train - Epoch 0 - batch 1000, batch loss is 0.095020, total loss is 596.753662
-----------------------
-----------------------
Train - Epoch 0 - batch 2000, batch loss is 0.144226, total loss is 1180.588135
-----------------------
-----------------------
Train - Epoch 0 - batch 3000, batch loss is 0.475690, total loss is 1753.656128
-----------------------
-----------------------
Train - Epoch 0 - batch 4000, batch loss is 0.966334, total loss is 2296.297119
-----------------------
-----------------------
Train - Epoch 0 - batch 5000, batch loss is 0.506560, total loss is 2831.590576
-----------------------
-----------------------
Epoch end: 26.656562328338623
At the end of epoch 0 loss is: 3166.66089
Validation loss: 720.43445 - F1 score: 0.88133 - precision: 0.7878359264497878 - recall: 1.0
-----------------------
-----------------------
Train - Epoch 1 - batch 1000, batch loss i

-----------------------
Train - Epoch 9 - batch 4000, batch loss is 0.867845, total loss is 531.864380
-----------------------
-----------------------
Train - Epoch 9 - batch 5000, batch loss is 0.000495, total loss is 644.074219
-----------------------
-----------------------
Epoch end: 19.400535583496094
At the end of epoch 9 loss is: 706.33209
Validation loss: 1584.23108 - F1 score: 0.80326 - precision: 0.8113553113553114 - recall: 0.7953321364452424
-----------------------
-----------------------
Train - Epoch 10 - batch 1000, batch loss is 0.000042, total loss is 108.928665
-----------------------
-----------------------
Train - Epoch 10 - batch 2000, batch loss is 0.123438, total loss is 226.042480
-----------------------
-----------------------
Train - Epoch 10 - batch 3000, batch loss is 0.004140, total loss is 332.445984
-----------------------
-----------------------
Train - Epoch 10 - batch 4000, batch loss is 0.352689, total loss is 475.136963
-----------------------
------

-----------------------
Train - Epoch 19 - batch 1000, batch loss is 0.004321, total loss is 7.774200
-----------------------
-----------------------
Train - Epoch 19 - batch 2000, batch loss is 0.000011, total loss is 16.115133
-----------------------
-----------------------
Train - Epoch 19 - batch 3000, batch loss is 0.000043, total loss is 22.293941
-----------------------
-----------------------
Train - Epoch 19 - batch 4000, batch loss is 0.000525, total loss is 29.373085
-----------------------
-----------------------
Train - Epoch 19 - batch 5000, batch loss is 0.000000, total loss is 33.315842
-----------------------
-----------------------
Epoch end: 20.677276849746704
At the end of epoch 19 loss is: 34.75623
Validation loss: 2704.28101 - F1 score: 0.81553 - precision: 0.8020833333333334 - recall: 0.829443447037702
-----------------------
-----------------------
Train - Epoch 20 - batch 1000, batch loss is 0.000038, total loss is 2.740980
-----------------------
-------------

...

-----------------------
Train - Epoch 13 - batch 1000, batch loss is 0.963434, total loss is 483.781036
-----------------------
-----------------------
Epoch end: 4.90337061882019
At the end of epoch 13 loss is: 689.30585
Validation loss: 202.01004 - F1 score: 0.79156 - precision: 0.7687804878048781 - recall: 0.8157349896480331
-----------------------
-----------------------
Train - Epoch 14 - batch 1000, batch loss is 0.962110, total loss is 476.605927
-----------------------
-----------------------
Epoch end: 5.0336010456085205
At the end of epoch 14 loss is: 679.21722
Validation loss: 201.97160 - F1 score: 0.79274 - precision: 0.7728613569321534 - recall: 0.8136645962732919
-----------------------
-----------------------
Train - Epoch 15 - batch 1000, batch loss is 0.956290, total loss is 469.553802
-----------------------
-----------------------
Epoch end: 5.074851751327515
At the end of epoch 15 loss is: 669.29608
Validation loss: 202.04308 - F1 score: 0.79213 - precision: 0.77263

-----------------------
Epoch end: 5.055541515350342
At the end of epoch 6 loss is: 936.73669
Validation loss: 230.94994 - F1 score: 0.81176 - precision: 0.6831683168316832 - recall: 1.0
-----------------------
----------
Starting straining for BS: 16 KS: 16 SS: 2 LR: 0.001
-----------------------
Epoch end: 5.597468852996826
At the end of epoch 0 loss is: 222.90074
Validation loss: 51.04972 - F1 score: 0.82173 - precision: 0.7122247532270312 - recall: 0.9710144927536232
-----------------------
-----------------------
Epoch end: 3.147801399230957
At the end of epoch 1 loss is: 206.33966
Validation loss: 50.60703 - F1 score: 0.80812 - precision: 0.7287853577371048 - recall: 0.906832298136646
-----------------------
-----------------------
Epoch end: 3.1292037963867188
At the end of epoch 2 loss is: 186.69554
Validation loss: 52.27648 - F1 score: 0.81138 - precision: 0.7287716405605935 - recall: 0.9151138716356108
-----------------------
-----------------------
Epoch end: 3.1554341316223

-----------------------
Epoch end: 3.145470380783081
At the end of epoch 5 loss is: 203.36433
Validation loss: 61.45196 - F1 score: 0.55938 - precision: 0.7495652173913043 - recall: 0.44616977225672877
-----------------------
-----------------------
Epoch end: 3.1437714099884033
At the end of epoch 6 loss is: 195.14552
Validation loss: 63.04604 - F1 score: 0.61290 - precision: 0.7647058823529411 - recall: 0.5113871635610766
-----------------------
-----------------------
Epoch end: 3.150735855102539
At the end of epoch 7 loss is: 186.91016
Validation loss: 68.65517 - F1 score: 0.66286 - precision: 0.7397959183673469 - recall: 0.6004140786749482
-----------------------
-----------------------
Epoch end: 3.1491427421569824
At the end of epoch 8 loss is: 179.34888
Validation loss: 72.74496 - F1 score: 0.62992 - precision: 0.7591240875912408 - recall: 0.5383022774327122
-----------------------
-----------------------
Epoch end: 3.1569085121154785
At the end of epoch 9 loss is: 172.76155
Va

-----------------------
Epoch end: 3.154304265975952
At the end of epoch 11 loss is: 156.31206
Validation loss: 54.54408 - F1 score: 0.73956 - precision: 0.788056206088993 - recall: 0.6966873706004141
-----------------------
-----------------------
Epoch end: 3.1813461780548096
At the end of epoch 12 loss is: 152.39349
Validation loss: 54.73770 - F1 score: 0.73997 - precision: 0.7889800703399765 - recall: 0.6966873706004141
-----------------------
-----------------------
Epoch end: 3.174351692199707
At the end of epoch 13 loss is: 148.50429
Validation loss: 55.04277 - F1 score: 0.73887 - precision: 0.7878077373974208 - recall: 0.6956521739130435
-----------------------
-----------------------
Epoch end: 3.1614553928375244
At the end of epoch 14 loss is: 144.79495
Validation loss: 55.35226 - F1 score: 0.73829 - precision: 0.7891637220259128 - recall: 0.6935817805383023
-----------------------
-----------------------
Epoch end: 3.161057472229004
At the end of epoch 15 loss is: 141.04089


KeyboardInterrupt: 

models_name                                     NaN
hash_id            d1db2b2ee1a633a19e12e22b079dbe27
val_f1                                     0.947368
val_precision                              0.918367
val_recall                                 0.978261
test_f1                                         NaN
test_precision                                  NaN
test_recall                                     NaN
epoch                                            12
mbti_trait                              introverted
fold                                              0
learning_rate                                 5e-06
batch_size                                        2
kernels_count                                    16
sentences_count                                   4
Name: 73, dtype: object

In [20]:
results

,models_name,hash_id,val_f1,val_precision,val_recall,test_f1,test_precision,test_recall,epoch,mbti_trait,fold,learning_rate,batch_size,kernels_count,sentences_count
0,NaN,e5a677ccc00e12a5dc84842655a04262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dfsfdsf,333,3,223333,1,13
1,NaN,e582184e3ad5cd8455f6487fd24aeb09,0.882376,0.789511,1.0,NaN,NaN,NaN,1.0,introverted,0,0.001,1,16,2
2,NaN,e0cbdb6e8d6c0a9e31da40e4bb9677d2,0.881329,0.787836,1.0,NaN,NaN,NaN,4.0,introverted,0,1e-07,1,16,2
3,NaN,ce44c7aa82fb4bd383c5ad2dcf0536a5,0.881329,0.787836,1.0,NaN,NaN,NaN,0.0,introverted,0,1e-08,1,16,2


/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


95       0.0
197      1.0
490      1.0
545      1.0
897      1.0
        ... 
9923     1.0
10009    1.0
10029    1.0
10075    1.0
10265    0.0
Name: introverted, Length: 67, dtype: float64

['-Afrodisiac-',
 '-Automaticity',
 '-Avacyn',
 '-BigSexy-',
 '-BlitzN9ne',
 '-CrestiaBell',
 '-Kaionic-',
 '-Maxis',
 '-SagaQ-',
 '-Yaldabaoth-',
 '-Yuuki-',
 '-blahblah',
 '-dyad-',
 '-godofwine-',
 '-insert_user-',
 '-kodoku-',
 '-komorebi',
 '-nox-',
 '-reggie-',
 '-taco',
 '-tactical-throw-away',
 '-y-y-',
 '00--0--00-',
 '006ahmed',
 '0b3r0n',
 '0de',
 '0jun',
 '0nn0',
 '0pointenergy',
 '0vermind74',
 '0x43617373696F6E',
 '100xyz',
 '101312',
 '10MeV',
 '110-115-120',
 '11sies_',
 '123askingquestions',
 '137288',
 '13segruts',
 '140IQ',
 '14royals',
 '17-03',
 '17b29a',
 '1mas',
 '1nc09n170',
 '1nf1n1te',
 '1o8',
 '1tspCreamOfAwesome',
 '20mcgug',
 '2154',
 '21stcenturyschizoidf',
 '22Starter22',
 '22vortex22',
 '247sleepy',
 '25taiku',
 '280394433708491',
 '2Dijit8',
 '2_4_16_256',
 '2bassguitarist2',
 '2ndXCharm',
 '2roamunseen',
 '2yang1001',
 '3062226abcd',
 '310outof365',
 '325Gerbils',
 '3ebfan',
 '3jt',
 '409latte',
 '420ZeusNoScope',
 '420pakalolo420',
 '46makun',
 '4ndr0